# LAB-04 

### Nombre: Blanco Moya Rosa Leonor

### Carrera: Ing. de Sistemas

# One VS All

Link del Dataset: https://data.world/uci/adult

Link del Repositorio de GitHub LAB-04: https://github.com/RosaCe21/Blanco-Moya-Rosa-Leonor-SIS420-AI/tree/ad429d2271697102d445377e6dd33c01dc08dedb/Laboratorios/LAB-04

# Informacióndel Dataset

Especificar cuales son sus  variables de entrada  (X): 

Variables de entrada son: 

X1:age: continuous.

X2:workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

X3: fnlwgt: continuous.

X4: education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

X5: education-num: continuous.

X6: marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

X7: occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

X8: relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

X9: race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

X10: sex: Female, Male.

X11: capital-gain: continuous.

X12: capital-loss: continuous.

X13: hours-per-week: continuous.

X14: native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

X15: class: >50K, <=50K

Y Label(Predicha):  Columna X6 "marital-status"


In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import optimize
from sklearn.preprocessing import PolynomialFeatures, RobustScaler
from sklearn.model_selection import train_test_split

%matplotlib inline

##Cargar el dataset

In [ ]:
data = pd.read_csv('./adult_data.csv')

##Procesamiento con Pandas

In [ ]:
X = data.iloc[:, :53].values
y = data.iloc[:, 54].values

y[y == 7] = 0

In [ ]:
print(X[0,:])
print(y)

[2596   51    3  258    0  510  221  232  148 6279    1    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    1    0    0    0    0    0    0    0    0    0    0]
[5 5 2 ... 3 3 3]


In [ ]:
X.shape

(581012, 53)

##Dividir el dataset en conjunto de entrenamiento (80%) y de prueba (20%)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape

((464809, 53), (116203, 53))

##Normalizacion de caracteristicas

In [ ]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

##Normalizar conjunto de prueba con los mismos parámetros

In [ ]:
X_train_norm, mu, sigma = featureNormalize(X_train)
X_test_norm = (X_test - mu) / sigma

In [ ]:
print(X_train_norm[0,:])
print(X_test_norm[0,:])
print(y_train)

[-1.84064495e+00 -1.35478696e+00  1.20680895e-01  9.37629885e-01
  3.51119576e+00 -8.54025270e-01 -4.92531212e-01 -6.73895992e-01
  1.17415734e-01 -7.42635855e-01 -9.03139057e-01 -2.33007037e-01
 -8.78410675e-01  3.83299804e+00 -7.25093310e-02 -1.14462336e-01
 -9.15336200e-02 -1.47513892e-01 -5.22188841e-02  9.34791597e+00
 -1.36824229e-02 -1.73576996e-02 -4.47270954e-02 -2.44228014e-01
 -1.47825727e-01 -2.33432647e-01 -1.75568599e-01 -3.18488196e-02
 -2.07433391e-03 -7.05186231e-02 -7.68640401e-02 -5.68807062e-02
 -8.33257507e-02 -1.26746963e-01 -3.81072657e-02 -2.46867855e-01
 -3.32492447e-01 -1.94500673e-01 -2.89413872e-02 -6.67532668e-02
 -4.33539565e-02 -4.06820129e-02 -4.97811305e-01 -2.33647781e-01
 -2.14750717e-01 -3.15084806e-01 -2.90670992e-01 -5.31428638e-02
 -5.77497310e-02 -1.42977903e-02 -2.27763368e-02 -1.66117671e-01
 -1.55152451e-01]
[-6.87092615e-01 -8.00692756e-01  3.87791844e-01 -2.80035178e-01
 -4.87901176e-01 -7.43738182e-01  8.16315395e-01 -9.77407688e-01
 -1.215

## Función Sigmoide

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

##Función de Costo regularizada para la regresión logistica

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)   # lo convierte a entero

    J = 0
    grad = np.zeros(theta.shape)  # creando los gradientes

    h = sigmoid(X.dot(theta.T))   # multiplica X por la transpuesta de theta

    temp = theta.copy()
    temp[0] = 0     # especifica que el primer elemento sea 0
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))  # h= y predicha

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

##Entrenamiento del modelo One-vs-All

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    m, n = X.shape   # verifica cuantas filas tiene
    all_theta = np.zeros((num_labels, n + 1))   # crea una matriz de ceros
    X = np.concatenate([np.ones((m, 1)), X], axis=1)    # Agrega unos a la matriz X

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1) # inicia fila de ceros pero con n+1
        options = {'maxiter': 150}  # utiliza 150 iteraciones
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='TNC',  # se puede utilizar diferentes metodos de optimización para converger mejor los datos
                                options=options)

        all_theta[c] = res.x

    return all_theta

## Prediccion One-vs-all

In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

Una vez que haya terminado, se llama a la función `predictOneVsAll` usando el valor aprendido de $\theta$. Debería apreciarse que la precisión del conjunto de entrenamiento es de aproximadamente 95,1% (es decir, clasifica correctamente el 95,1% de los ejemplos del conjunto de entrenamiento).

##Grafica de costos durante el entrenamiento

In [ ]:
def plot_cost(all_theta, X, y, lambda_):
    m = X.shape[0]
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    costs = []

    for i in range(1, 1500):  # Se itera para simular el costo por iteraciones
        J, grad = lrCostFunction(all_theta[0], X, (y == 0).astype(int), lambda_)
        costs.append(J)

    plt.plot(range(1, 1500), costs)
    plt.xlabel('Iteraciones')
    plt.ylabel('Costo')
    plt.title('Función de Costo durante el entrenamiento')
    plt.show()

##Parámetros iniciales

In [ ]:
lambda_ = 0.1
num_labels = 7    # Dependiendo el numero de clases del dataset

## Entrenar el Modelo

In [ ]:
all_theta = oneVsAll(X_train_norm, y_train, num_labels, lambda_)

<ipython-input-71-dea8621dc514>:10: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(lrCostFunction,
<ipython-input-70-cc496eb8c808>:16: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))  # h= y predicha


## Realizar predicciones en el conjunto de prueba

In [ ]:
pred_train = predictOneVsAll(all_theta, X_train_norm)
train_exactitud = np.mean(pred_train == y_train) * 100
print(f'Precisión del conjunto de entrenamiento: {train_exactitud:.2f}%')

pred_test = predictOneVsAll(all_theta, X_test_norm)
test_exactitud = np.mean(pred_test == y_test) * 100
print(f'Precisión del conjunto de prueba: {test_exactitud:.2f}%')

Precisión del conjunto de entrenamiento: 71.51%
Precisión del conjunto de prueba: 71.54%
